In [431]:
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn import preprocessing
import matplotlib.pyplot as plt
from copy import deepcopy
from sklearn import tree
import rasterio as rio
import pandas as pd
import numpy as np
import importlib
import sklearn
import random
import pickle
import torch
import math
import copy

import resnet18
importlib.reload(resnet18)
from helpers import *

import warnings
warnings.filterwarnings('ignore')

In [339]:
d = dataLoader("./final_pics", "./us_migration.csv")

In [340]:
x_train, y_train, x_val, y_val = d.train_val_split(split = .80)

In [432]:
#y - 'number_moved'
#x - 'everything else that is or can be represented as a float.'

####### Build and fit the Model
lr = 1e-5
batchSize = 50
model = resnet18.resnet18(outDim = batchSize)

In [433]:
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr = lr)

In [434]:
def train_model(model, criterion, optimizer, lr, batchSize, num_epochs):

    losses = []
    maes = []
    best_mae = 100000000
    i_to_print = torch.tensor([0,1,2,3,4])

    for t in range(num_epochs):

        for phase in ['train', 'val']:

            if phase == 'train':

                #Batches
                batchObs = random.sample(range(0, len(y_train)), batchSize)
                # print(batchObs)
                modelX = [x_train[i] for i in batchObs]
                modelX = torch.tensor(list(modelX), requires_grad = True, dtype = torch.float32)
                modely = torch.tensor([y_train[i] for i in batchObs], dtype = torch.float32)  # MADE A CHANGE HERE 
                y_pred = model(modelX, t)
                # print(y_pred)
                
                loss = criterion(y_pred, modely)  
                
                # Zero gradients, perform a backward pass, and update the weights.
                optimizer.zero_grad()
                grad = torch.autograd.grad(outputs=loss, inputs=modelX, retain_graph = True)
                # print("    GRADIENT: ", grad[0][0].shape)
                # print("    GRADIENT: ", grad[0])
                loss.backward()
                optimizer.step()
                # https://discuss.pytorch.org/t/updatation-of-parameters-without-using-optimizer-step/34244/4
                # with torch.no_grad():
                #     for name, p in model.named_parameters():
                #         if name == 'SocialSig.W':
                #         # print("    In with:        ", p.data)
                #             new_val = socialSigLayers.update_function(p, grad[0], loss, lr)
                #         # print("NEW WEIGHTS: ", new_val)
                #             p.copy_(new_val)

                print("EPOCH: ", t)
                # print(modely)
                epoch_mae = mae(y_pred, modely).item()
                print("    TRAIN    Loss:   ", loss.item(), "     MAE:   ", epoch_mae)
                # print("    Train Preds:     ", torch.index_select(y_pred, 0, i_to_print))
                # print("    Train True:       ", torch.index_select(modely, 0, i_to_print))

            if phase == 'val':

                #Batches
                batchObs = random.sample(range(0, len(y_train)), batchSize)
                # print(batchObs)
                modelX = [x_train[i] for i in batchObs]
                modelX = torch.tensor(list(modelX), requires_grad = True, dtype = torch.float32)
                modely = torch.tensor([y_train[i] for i in batchObs], dtype = torch.float32)  # MADE A CHANGE HERE 
                
                # Perform evaluations of the batch predictions
                y_pred = model(modelX, t)
                
                loss = criterion(y_pred, modely)  
                epoch_mae = mae(y_pred, modely).item()
                print("    VAL      Loss:   ", loss.item(), "     MAE:   ", epoch_mae)
                # print("    Val Preds:       ", torch.index_select(y_pred, 0, i_to_print))
                # print("    Val True:       ", torch.index_select(modely, 0, i_to_print))

                if epoch_mae < best_mae:
                    
                    best_mae = epoch_mae
                    best_model_wts = copy.deepcopy(model.state_dict())
                    model_name = './models/test_Epoch' + str(t) + '.sav'
                    pickle.dump(model, open(model_name, 'wb'))

                losses.append(loss.item())
                maes.append(epoch_mae)
                
        print("\n")

    print("TRAINING COMPLETE")
    print("Best MAE: ", best_mae)

    # print(best_model_wts)

    # return model.load_state_dict(best_model_wts)
    return best_model_wts, losses, maes, y_pred, modely

In [429]:
num_epochs = 100

best_model_wts, losses, maes, ypreds, ytrue = train_model(model, criterion, optimizer, lr, batchSize, num_epochs)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x1254400 and 250880x10)

In [418]:
model.load_state_dict(best_model_wts)

<All keys matched successfully>

In [419]:
def pred_dataset(x_val, model, batch_size, y_val):
    preds, true_vals = [], []
    lst = [i for i in range(0, len(x_val))]
    batches = [lst[i:i+batch_size] for i in range(0, len(lst), batch_size)]
    batches = [i for i in batches if len(i) == batch_size]
    n = len(batches)
    for subset in range(0, n):
        cur_xval = [x_val[i] for i in batches[subset]]
        modelX = torch.tensor(list(cur_xval), requires_grad = True, dtype = torch.float32)
        cur_preds = model(modelX, 400)
        preds.append(list(cur_preds.detach().numpy()))
        true_vals.append([y_val[i] for i in batches[subset]])

        print("Done with batch ", str(subset), " out of ", str(n))

    true_vals = [item for sublist in true_vals for item in sublist]
    preds = [item for sublist in preds for item in sublist]

    to_ret = pd.DataFrame()
    to_ret['id'] = [i for i in range(0, len(true_vals))]
    to_ret['pred'] = preds
    to_ret['true_val'] = true_vals
    
    return to_ret

In [420]:
preds_df = pred_dataset(x_val, model, 10, y_val)
preds_df.head()

Done with batch  0  out of  46
Done with batch  1  out of  46
Done with batch  2  out of  46
Done with batch  3  out of  46
Done with batch  4  out of  46
Done with batch  5  out of  46
Done with batch  6  out of  46
Done with batch  7  out of  46
Done with batch  8  out of  46
Done with batch  9  out of  46
Done with batch  10  out of  46
Done with batch  11  out of  46
Done with batch  12  out of  46
Done with batch  13  out of  46
Done with batch  14  out of  46
Done with batch  15  out of  46
Done with batch  16  out of  46
Done with batch  17  out of  46
Done with batch  18  out of  46
Done with batch  19  out of  46
Done with batch  20  out of  46
Done with batch  21  out of  46
Done with batch  22  out of  46
Done with batch  23  out of  46
Done with batch  24  out of  46
Done with batch  25  out of  46
Done with batch  26  out of  46
Done with batch  27  out of  46
Done with batch  28  out of  46
Done with batch  29  out of  46
Done with batch  30  out of  46
Done with batch  3

,id,pred,true_val
0,0,131.377502,84
1,1,57.921864,619
2,2,150.939316,99
3,3,98.996971,335
4,4,91.957092,242


In [421]:
allocation_error = abs((sum(preds_df['true_val']) - sum(preds_df['pred'])) / len(preds_df['pred']))
allocation_error

118.9601009617681

In [422]:
quantity_error = abs(sum(preds_df['true_val']) - sum(preds_df['pred']))
quantity_error

54721.64644241333

In [423]:
mae = mean_absolute_error(preds_df['true_val'], preds_df['pred'])
mae

215.12865516828455

In [424]:
mape = mean_absolute_percentage_error(preds_df['true_val'], preds_df['pred'])
mape

1.4757216542315956e+16

In [425]:
r2 = r2_score(preds_df['true_val'], preds_df['pred'])
r2

-0.1594528511954303